In [630]:
import pandas as pd
import numpy as np
import vnstock as vn
import calculation as cal

In [631]:
data = cal.DataProcessor.load_data('VCB') 
# data = data.set_index('time')
#Tính trung bình 20 phiên gần nhất
mean_20 = data['volume'].rolling(window=20).mean() 
#Tính on-balance volume
data['on-balance_volume'] = data['volume']/mean_20 

data['OBV_label'] = data['on-balance_volume'].apply(cal.DataProcessor.label_values)
#Giá đóng cửa ngày t-1
data['close_t_minus_1'] = data['close'].shift(1) 
data['bar_type'] = data.apply(cal.DataProcessor.compare_close_prices, axis=1)   
data['para'] =abs(data['close'] - data['open'])/(data['high'] - data['low'])

data['label_spread'] = data['para'].apply(cal.DataProcessor.label_spread)
data['close_bar_label'] = data.apply(cal.DataProcessor.label_close_bar, axis=1)


In [632]:

# df_filtered = data[data['bar_type'] == 'up-bar']
# df_filtered


# Pattern of Weakness

## Weakness A
- down bar
- high volume
- medium or low spread
- close: middle-third or bottom-third

In [633]:
def is_weakness_a_signal(data):
    if (data['bar_type'] == 'down-bar') and \
        (data['OBV_label']=='high') and \
        ((data['label_spread'] == 'low') or (data['label_spread']=='medium')) and \
        ((data['close_bar_label'] == 'middle-third') or (data['close_bar_label']=='bottom-third')):
        return True
    else:
        return False

In [634]:
# Sample data
sample_data = {
    'bar_type': 'down-bar',
    'OBV_label': 'high',
    'label_spread': 'low',
    'close_bar_label': 'middle-third'
}
result = is_weakness_a_signal(sample_data)
print("Is weakness a signal?", result)

Is weakness a signal? True


## No demand - Weakness B
- up bar
- volume: low or high
- spread: low
- close: bottom-third and middle-third

In [635]:
def is_no_demand_signal(data):
    if (data['bar_type'] == 'up-bar') and \
        ((data['OBV_label']=='high') or (data['OBV_label']=='low')) and \
        (data['label_spread'] == 'low') and \
        ((data['close_bar_label'] == 'bottom-third') or data['close_bar_label'] == 'middle-third'):
        return True
    else:
        return False

In [636]:
result = is_no_demand_signal(sample_data)
print("Is no demand signal?", result)

Is no demand signal? False


## Up-trust - Pseudo Up-trust
- up bar or down bar 
- high spread
- close: bottom-third
- high or low volume


In [637]:
def is_up_trust_signal(data):
    if ((data['OBV_label']=='high') or (data['OBV_label']=='low')) and \
        (data['label_spread'] == 'high') and \
        ((data['close_bar_label'] == 'bottom-third') ):
        return True
    else:
        return False

## Stop volume
- up bar
- high spread
- high volume

In [638]:
def is_stop_volume_signal(data):
    if (data['bar_type']=='up-bar') and \
        (data['OBV_label']=='high') and \
        (data['label_spread'] == 'high'):
        return True
    else:
        return False

In [639]:
for i in range(len(data)):
    # Lấy ngày tương ứng
    date = data.loc[i, 'time']
    
    # Áp dụng hàm tương ứng cho ngày tương ứng
    if i % 4 == 0:
        data.loc[i, 'results'] = is_weakness_a_signal(data.loc[i])
    elif i % 4 == 1:
        data.loc[i, 'results'] = is_no_demand_signal(data.loc[i])
    elif i % 4 == 2:
        data.loc[i, 'results'] = is_up_trust_signal(data.loc[i])
    elif i % 4 == 3:
        data.loc[i, 'results'] = is_stop_volume_signal(data.loc[i])
    
data.head(10)

,time,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread,close_bar_label,results
0,2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium,bottom-third,False
1,2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low,middle-third,False
2,2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium,top-third,False
3,2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high,bottom-third,False
4,2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high,top-third,False
5,2019-01-09,35780,36170,35330,35980,1136370,VCB,NaN,NaN,35460.0,up-bar,0.238095,low,top-third,False
6,2019-01-10,35980,35980,35460,35460,794870,VCB,NaN,NaN,35980.0,down-bar,1.000000,high,bottom-third,False
7,2019-01-11,35590,35780,35400,35590,1115410,VCB,NaN,NaN,35460.0,up-bar,0.000000,high,middle-third,False
8,2019-01-14,35590,35780,35400,35660,759980,VCB,NaN,NaN,35590.0,up-bar,0.184211,low,top-third,False
9,2019-01-15,35660,36170,35590,36040,1071080,VCB,NaN,NaN,35660.0,up-bar,0.655172,medium,top-third,False


In [640]:
abc = data[data['results'] == True]
abc

,time,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread,close_bar_label,results
23,2019-02-11,36750,37530,36620,37460,1395030,VCB,1.641787,high,36430.0,up-bar,0.780220,high,top-third,True
36,2019-02-28,39910,40300,39010,39010,2027080,VCB,1.438314,high,39910.0,down-bar,0.697674,medium,bottom-third,True
80,2019-05-07,42750,43200,41780,42300,1103060,VCB,1.798570,high,42750.0,down-bar,0.316901,medium,middle-third,True
97,2019-05-30,43520,43910,43390,43650,320900,VCB,0.543429,low,43520.0,up-bar,0.250000,low,middle-third,True
158,2019-08-23,51580,51710,51070,51070,451500,VCB,0.614727,low,51580.0,down-bar,0.796875,high,bottom-third,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,2023-09-15,88900,89800,88600,89800,2142400,VCB,1.284774,high,89100.0,up-bar,0.750000,high,top-third,True
1186,2023-10-02,87100,87200,86500,86500,765700,VCB,0.528415,low,87500.0,down-bar,0.857143,high,bottom-third,True
1204,2023-10-26,85000,86000,83500,84500,1420300,VCB,1.688301,high,85800.0,down-bar,0.200000,low,middle-third,True
1236,2023-12-11,85000,85300,84100,84200,1400000,VCB,1.257619,high,85000.0,down-bar,0.666667,medium,bottom-third,True


In [641]:
data['results'].value_counts()

results
False    1211
True       63
Name: count, dtype: int64

In [642]:

# Tạo cột "Lệnh"
data['execute_a_trade'] = 'hold'  # Đặt mặc định là "Hold"

window_size =4  # Kích thước cửa sổ

data['rolling_sum'] = data['results'].rolling(window_size).sum()

data.loc[data['rolling_sum'] == window_size, 'execute_a_trade'] = 'buy'

data = data.drop(columns='rolling_sum')

data.head(10)

,time,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread,close_bar_label,results,execute_a_trade
0,2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium,bottom-third,False,hold
1,2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low,middle-third,False,hold
2,2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium,top-third,False,hold
3,2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high,bottom-third,False,hold
4,2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high,top-third,False,hold
5,2019-01-09,35780,36170,35330,35980,1136370,VCB,NaN,NaN,35460.0,up-bar,0.238095,low,top-third,False,hold
6,2019-01-10,35980,35980,35460,35460,794870,VCB,NaN,NaN,35980.0,down-bar,1.000000,high,bottom-third,False,hold
7,2019-01-11,35590,35780,35400,35590,1115410,VCB,NaN,NaN,35460.0,up-bar,0.000000,high,middle-third,False,hold
8,2019-01-14,35590,35780,35400,35660,759980,VCB,NaN,NaN,35590.0,up-bar,0.184211,low,top-third,False,hold
9,2019-01-15,35660,36170,35590,36040,1071080,VCB,NaN,NaN,35660.0,up-bar,0.655172,medium,top-third,False,hold


In [652]:
# import pandas as pd

# # Tạo DataFrame mẫu
# data1 = {'ngay': ['2024-02-25', '2024-02-26', '2024-02-27', '2024-02-28', '2024-03-01', '2024-03-02','2024-03-03', '2024-03-04', '2024-03-05'],
#         'results': [False, True, True, True, True, True, False, True, True]}
# df = pd.DataFrame(data1)
# # Tạo cột "Lệnh"
# df['execute_a_trade'] = 'hold'  # Đặt mặc định là "Hold"

# window_size = 4  # Kích thước cửa sổ

# df['rolling_sum'] = df['results'].rolling(window_size).sum()

# df.loc[df['rolling_sum'] == window_size, 'execute_a_trade'] = 'buy'

# df = df.drop(columns='rolling_sum')

# print(df)

         ngay  results execute_a_trade
0  2024-02-25    False            hold
1  2024-02-26     True            hold
2  2024-02-27     True            hold
3  2024-02-28     True            hold
4  2024-03-01     True             buy
5  2024-03-02     True             buy
6  2024-03-03    False            hold
7  2024-03-04     True            hold
8  2024-03-05     True            hold


In [644]:
data['execute_a_trade'] = 'hold'

# Kiểm tra 4 ngày liên tiếp và gán giá trị 'buy' cho cột mới nếu điều kiện đúng
for i in range(len(data) - 3):
    if all(data['results'].iloc[i:i+4]):
        data.loc[i:i+3, 'action'] = 'buy'

In [645]:
data[['time', 'OBV_label', 'bar_type', 'label_spread', 'close_bar_label', 'results', 'execute_a_trade']].head(20)

,time,OBV_label,bar_type,label_spread,close_bar_label,results,execute_a_trade
0,2019-01-02,NaN,down-bar,medium,bottom-third,False,hold
1,2019-01-03,NaN,down-bar,low,middle-third,False,hold
2,2019-01-04,NaN,up-bar,medium,top-third,False,hold
3,2019-01-07,NaN,up-bar,high,bottom-third,False,hold
4,2019-01-08,NaN,up-bar,high,top-third,False,hold
5,2019-01-09,NaN,up-bar,low,top-third,False,hold
6,2019-01-10,NaN,down-bar,high,bottom-third,False,hold
7,2019-01-11,NaN,up-bar,high,middle-third,False,hold
8,2019-01-14,NaN,up-bar,low,top-third,False,hold
9,2019-01-15,NaN,up-bar,medium,top-third,False,hold


In [646]:
data['execute_a_trade'].value_counts()

execute_a_trade
hold    1274
Name: count, dtype: int64

# Power pattern

## Power A
- up bar
- medium spread
- medium or high volume
- close: top-third

In [647]:
def is_power_A_signal(data):
    if (data['bar_type'] == 'up-bar') and \
        (data['label_spread'] == 'medium') and \
        ((data['OBV_label']=='medium') or (data['OBV_label'] == 'low')) and \
        (data['close_bar_label'] == 'top-third'):
        return True
    else:
        return False

## Power B - Lack of order
- down bar 
- low spread 
- low or high volume
- close: top-third or bottom-third

In [648]:
def is_power_B_signal(data):
    if (data['bar_type'] == 'down-bar') and \
        (data['label_spread'] == 'low') and \
        ((data['OBV_label']=='low') or (data['OBV_label'] == 'high')) and \
        ((data['close_bar_label'] == 'top-third') or (data['close_bar_label'] == 'bottom-third')):
        return True
    else:
        return False

## Reverse Up-Trust - Pseudo Up-trust
- up bar or down bar 
- high spread
- close: bottom-third
- high and low volume


In [649]:
def is_reverse_up_trust_signal(data):
    if ((data['bar_type'] == 'down-bar') or (data['bar_type']) == 'up-bar') and \
        (data['label_spread'] == 'high') and \
        ((data['OBV_label']=='low') or (data['OBV_label'] == 'high')) and \
        (data['close_bar_label'] == 'bottom-third'):
        return True
    else:
        return False

## Stopped volume
- down bar
- low spread
- close: middle-third
- high volume


In [650]:
def is_stopped_volume_signal(data):
    if (data['bar_type'] == 'down-bar') and \
        (data['label_spread'] == 'low') and \
        (data['OBV_label'] == 'high') and \
        (data['close_bar_label'] == 'middle-third'):
        return True
    else:
        return False